In [1]:
import json 
import os

def read_jsonl(path:str):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f.readlines() if  line]


In [2]:
path = os.path.join("../Data/GSM8K/data/test.jsonl")
test_data = read_jsonl(path)

In [3]:
import warnings
warnings.filterwarnings("ignore")
import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
import torch 
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer=AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
# model=AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct").to(device)

tokenizer=AutoTokenizer.from_pretrained("../Record/Qwen2-0.5B-Instruct")
model=AutoModelForCausalLM.from_pretrained("../Record/Qwen2-0.5B-Instruct").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
print(test_data[0].keys())

dict_keys(['question', 'answer'])


In [6]:
predictions = []
for idx,question in enumerate(test_data):
    eight_shot="Question: Bert, Ernie, and Peggy collect stamps.  Bert has four times as many stamps as Ernie, but Ernie has three times as many stamps as Peggy.  If Peggy currently has 75 stamps in her collection, how many stamps does she need to add to her collection to have a collection as large as Bert's collection?\nAnswer: Ernie has three times as many stamps as Peggy, or 3*75=<<3*75=225>>225 stamps.\nBert has 4 times as many stamps as Ernie, or 4*225=<<4*225=900>>900 stamps.\nThus, Peggy would need 900-75=<<900-75=825>>825 more stamps to have a collection as large as Bert's.\n#### 825\n\nQuestion: Kurt's old refrigerator cost $0.85 a day in electricity.  He recently bought a new energy-efficient refrigerator that only cost $0.45 a day in electricity.  How much money does Kurt save in a 30-day month with his new refrigerator?\nAnswer: The old refrigerator costs $0.85 a day so for 30 days it costs .85*30 = $<<0.85*30=25.50>>25.50\nThe new refrigerator costs $0.45 a day so for 30 days it costs .45*30 =$<<0.45*30=13.50>>13.50\nOver 30 days, Kurt saves 25.50-13.50 = $<<25.50-13.50=12.00>>12.00 a month with his new refrigerator\n#### 12\n\nQuestion: Angel has written letters to all of her pen pals and is putting all of the letters in envelopes. Most of the letters are put into large envelopes and the remaining 20 letters are put into small envelopes. The large envelopes each contain 2 letters each. If Angel has written a total of 80 letters, how many large envelopes did she use?\nAnswer: As some of the letters are in small envelopes, there must be 80 total letters – 20 letters in small envelopes = <<80-20=60>>60 letters in large envelopes.\nAngel therefore used 60 letters / 2 letters per large envelope = <<60/2=30>>30 large envelopes.\n#### 30\n\nQuestion: Last week, Charles bought a t-shirt for $30, a backpack for $10, and a blue cap for $5. The store gave him a discount and, in total, he only spent $43. How much of a discount did they give him, in dollars?\nAnswer: The purchases he made totaled $30 + $10 + $5 = $<<30+10+5=45>>45\nHe should pay $45, but he only paid $43, so $45 - $43 = $<<45-43=2>>2 was the discount they gave him\n#### 2\n\nQuestion: Tim has 30 less apples than Martha, and Harry has half as many apples as Tim. If Martha has 68 apples, how many apples does Harry have?\nAnswer: Tim has 68-30 = <<68-30=38>>38 apples.\nHarry has 38/2 = <<38/2=19>>19 apples.\n#### 19\n\nQuestion: A curry house sells curries that have varying levels of spice. Recently, a lot of the customers have been ordering very mild curries and the chefs have been having to throw away some wasted ingredients. To reduce cost and food wastage, the curry house starts monitoring how many ingredients are actually being used and changes their spending accordingly. The curry house needs 3 peppers for very spicy curries, 2 peppers for spicy curries, and only 1 pepper for mild curries. After adjusting their purchasing, the curry house now buys the exact amount of peppers they need. Previously, the curry house was buying enough peppers for 30 very spicy curries, 30 spicy curries, and 10 mild curries. They now buy enough peppers for 15 spicy curries and 90 mild curries. They no longer sell very spicy curries. How many fewer peppers does the curry house now buy?\nAnswer: The curry house previously bought 3 peppers per very spicy curry * 30 very spicy curries = <<3*30=90>>90 peppers for very spicy curries.\nThey also bought 2 peppers per spicy curry * 30 spicy curries = <<2*30=60>>60 peppers for spicy curries.\nThey also bought 1 pepper per mild curry * 10 mild curries = <<1*10=10>>10 peppers for mild curries.\nSo they were previously buying 90 + 60 + 10 = <<90+60+10=160>>160 peppers.\nThey now buy 2 peppers per spicy curry * 15 spicy curries = <<2*15=30>>30 peppers for spicy curries.\nThey also now buy 1 pepper per mild curry * 90 mild curries = <<1*90=90>>90 peppers for mild curries.\nSo they now buy 30 + 90 = <<30+90=120>>120 peppers.\nThis is a difference of 160 peppers bought originally - 120 peppers bought now = <<160-120=40>>40 peppers.\n#### 40\n\nQuestion: Mary is chopping up some old furniture to make firewood. Chairs make 6 sticks of wood, tables make 9 sticks of wood, and stools make 2 sticks of wood. Mary needs to burn 5 sticks of wood per hour to stay warm. If Mary chops up 18 chairs, 6 tables and 4 stools, how many hours can she keep warm?\nAnswer: First find the total sticks from the chairs: 6 sticks/chair * 18 chairs = <<6*18=108>>108 sticks\nThen find the total sticks from the tables: 9 sticks/table * 6 tables = <<9*6=54>>54 sticks\nThen find the total sticks from the stools: 2 sticks/table * 4 stools = <<2*4=8>>8 sticks\nThen add up the total number of sticks she gets: 108 sticks + 54 sticks + 8 sticks = <<108+54+8=170>>170 sticks\nThen divide the total number of sticks by the number of sticks she burns per hour to find the total number of hours she can keep warm: 170 sticks / 5 sticks/hour = <<170/5=34>>34 hours\n#### 34\n\nQuestion: Nancy's ex owes her child support. He's supposed to pay 30% of his income each year. For 3 years, he made $30,000/year, then he got a 20% raise for the next four years. If he's only ever paid $1,200, how much does he owe her?\nAnswer: First find how much money the ex made during the first 3 years by multiplying his annual earnings by the number of years: 3 years * $30,000/year = $<<3*30000=90000>>90,000/\nThen find how much his raise was by multiplying his salary by the percentage raise: $30,000/year * 20% = $<<30000*20*.01=6000>>6,000/year\nThen add the raise to his original salary to find his new salary: $6,000/year + $30,000/year = $<<6000+30000=36000>>36,000/year\nThen find how much he made working at the new salary by multiplying it by the 4 years he worked: 4 years * $36,000/year = $<<4*36000=144000>>144,000\nAdd his earnings from the three years and the four years to find his total earnings: $144,000 + $90,000 = $<<144000+90000=234000>>234,000\nNow multiply his total earnings by 30% to find how much he was supposed to pay: $234,000 * 30% = $<<234000*30*.01=70200>>70,200\nNow subtract the amount he paid from this number to find how much he owes: $70,200 - $1,200 = $<<70200-1200=69000>>69,000\n#### 69000\n\nQuestion: Leila buys 3 cucumbers from the market. Cucumbers are $2 each. Jack buys 5 tomatoes from the grocery store.  Tomatoes are $1 each. Chase buys 1 head of lettuce from the farmer’s market.  Lettuce cost $3 each. Together, how much did the three of them spend to make a salad for the potluck?\nAnswer:"
    msg=[
    {"role": "user","content": f"{eight_shot}Question:{question['question']}\n\nAnswer:"},
    ]
    text = tokenizer.apply_chat_template(
        msg,
        tokenize=False,
        add_generation_prompt=True
    )
    print(text)
    break
    model_inputs = tokenizer([text], padding=True, return_tensors="pt",truncation=True).to(device)
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    predictions.append(response)
# with open('data/predictions.txt', 'w') as f:
#     f.write("\n".join(predictions))

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Question: Bert, Ernie, and Peggy collect stamps.  Bert has four times as many stamps as Ernie, but Ernie has three times as many stamps as Peggy.  If Peggy currently has 75 stamps in her collection, how many stamps does she need to add to her collection to have a collection as large as Bert's collection?
Answer: Ernie has three times as many stamps as Peggy, or 3*75=<<3*75=225>>225 stamps.
Bert has 4 times as many stamps as Ernie, or 4*225=<<4*225=900>>900 stamps.
Thus, Peggy would need 900-75=<<900-75=825>>825 more stamps to have a collection as large as Bert's.
#### 825

Question: Kurt's old refrigerator cost $0.85 a day in electricity.  He recently bought a new energy-efficient refrigerator that only cost $0.45 a day in electricity.  How much money does Kurt save in a 30-day month with his new refrigerator?
Answer: The old refrigerator costs $0.85 a day so for 30 days it costs .85*30 = $<<0.85*30=25.50>>25.50

In [17]:
import re 
def test(predictions,test_data):
    acc=0
    for i in range(len(test_data)):
        ans=re.findall(r'\d+',predictions[i])[-1]
        label=re.findall(r'\d+',test_data[i]['answer'])[-1]
        if ans==label:
            acc+=1
        else:
            print(f"Ans:{ans}\t Label:{label}")
            print(f"Response:{predictions[i]}")
            print(f"Label:{test_data[i]['answer']}")
        if (i+1)%5==0:
            print(f"{i+1}/{len(test_data)} Accuracy:{acc/i+1}")
    return acc/len(test_data)
accuracy=test(predictions,test_data)
print(accuracy)

Ans:54	 Label:3
Response:Leila buys 3 cucumbers and Jack buys 5 tomatoes, which makes a total of \(3+5=8\) items. Since each item consists of 2 bolts of blue fiber and half that amount (which is \(2/2=1\)) of white fiber, we calculate the total fibers needed as follows:

\[2 \text{ bolts} \times 8 \text{ items} = 16 \text{ bolts}\]

Chase buys 1 head of lettuce, which requires one bolt of blue fiber.

Therefore, the total amount of fabric required for the salad is \(16 \text{ bolts} + 1 \text{ bolt} + 1 \text{ bolt} = 18 \text{ bolts}\).

Since each bolt costs $3, the total cost is \(18 \text{ bolts} \times \$3/\text{bolt} = \$54\).

So, together, the three of them spent \$54 to make the salad for the potluck.
#### 54
Label:It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3
